In [35]:
import pandas as pd
import numpy as np
import copy

import warnings
warnings.filterwarnings('ignore')

# Read data

In [36]:
dt = 'tr'
path = f'./data/{dt}.csv'
df = pd.read_csv(path)

In [37]:
df['LabelStr'] = df['Label']
df['Label'] = 0
df.loc[df['LabelStr'].str.contains('To-Backgro'), 'Label'] = 1
df.loc[df['LabelStr'].str.contains('From-Backg'), 'Label'] = 2
df.loc[df['LabelStr'].str.contains('From-Norma'), 'Label'] = 3
df.loc[df['LabelStr'].str.contains('To-Normal-'), 'Label'] = 4
df.loc[df['LabelStr'].str.contains('From-Botne'), 'Label'] = 5
df.loc[df['LabelStr'].str.contains('Normal-'), 'Label'] = 3

In [38]:
df['Conversation'] = df['SrcAddr']+' -> '+df['DstAddr']

In [39]:
window_width = 7200
window_stride = 3600

df['StartTimeInt'] = pd.to_datetime(df['StartTime']).astype(np.int64)*1e-9
datetime_start = df['StartTimeInt'].min()

df['Window_lower'] = (df['StartTimeInt'] - datetime_start - window_width) / window_stride + 1
df['Window_lower'].clip(lower=0, inplace=True)
df['Window_upper_excl'] = (df['StartTimeInt'] - datetime_start) / window_stride + 1
df = df.astype({'Window_lower': int, 'Window_upper_excl': int})

df.sort_values(by=['Window_lower'])

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,...,dTos,TotPkts,TotBytes,SrcBytes,Label,LabelStr,Conversation,StartTimeInt,Window_lower,Window_upper_excl
0,1,2022-07-27 02:24:40.822889,0.000298,tcp,68.55.208.148,53678,->,150.35.88.103,30966,S_RA,...,0.0,1,126,65,0,flow=Background-TCP-Attempt,68.55.208.148 -> 150.35.88.103,1.658889e+09,0,1
66458,66459,2022-07-27 03:24:59.896836,0.000187,udp,150.35.88.28,40370,<->,150.35.83.12,53,CON,...,0.0,2,204,65,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.28 -> 150.35.83.12,1.658892e+09,0,2
66457,66458,2022-07-27 03:24:59.896300,0.000353,udp,150.35.88.28,54942,<->,150.35.83.12,53,CON,...,0.0,2,208,79,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.28 -> 150.35.83.12,1.658892e+09,0,2
66456,66457,2022-07-27 03:24:59.865859,0.000177,udp,150.35.88.28,41396,<->,150.35.83.12,53,CON,...,0.0,2,206,66,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.28 -> 150.35.83.12,1.658892e+09,0,2
66455,66456,2022-07-27 03:24:59.865482,0.000190,udp,150.35.88.28,54214,<->,150.35.83.12,53,CON,...,0.0,2,207,78,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.28 -> 150.35.83.12,1.658892e+09,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421008,2421009,2022-07-28 17:45:57.965706,0.000271,udp,150.35.88.63,36863,<->,150.35.83.12,53,CON,...,0.0,1,572,79,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.63 -> 150.35.83.12,1.659030e+09,38,40
2421007,2421008,2022-07-28 17:45:57.965402,0.000410,udp,150.35.88.63,50706,<->,150.35.83.12,53,CON,...,0.0,1,603,78,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.63 -> 150.35.83.12,1.659030e+09,38,40
2421006,2421007,2022-07-28 17:45:57.965255,0.000353,udp,150.35.88.63,33103,<->,150.35.83.12,53,CON,...,0.0,1,401,77,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.63 -> 150.35.83.12,1.659030e+09,38,40
2421020,2421021,2022-07-28 17:45:57.970107,0.000402,udp,150.35.88.63,50434,<->,150.35.83.12,53,CON,...,0.0,2,243,70,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.88.63 -> 150.35.83.12,1.659030e+09,38,40


In [40]:
nb_windows = df['Window_upper_excl'].max()
print(df['Label'].value_counts())
print(df['Label'].unique())
print()
print(df[df['Label'] == 5]['SrcAddr'].unique())
print(df[df['Label'] == 5]['Window_lower'].unique())

0    1598860
1     814569
3      71192
5      26747
2        507
Name: Label, dtype: int64
[0 1 3 2 5]

['150.35.87.168']
[ 2  3  4 25 26 27 28 29 30 31 32 33 34 35 36 37 38]


In [41]:
df.loc[df['Window_lower'] == 3]

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,...,dTos,TotPkts,TotBytes,SrcBytes,Label,LabelStr,Conversation,StartTimeInt,Window_lower,Window_upper_excl
220697,220698,2022-07-27 06:24:40.948885,1.691439,tcp,150.35.87.168,3031,->,127.149.234.48,22,S_RA,...,0.0,5,366,186,5,flow=From-Botnet-V44-TCP-Attempt,150.35.87.168 -> 127.149.234.48,1.658903e+09,3,5
220698,220699,2022-07-27 06:24:41.228201,8.928617,tcp,150.35.87.168,3032,->,127.149.234.49,22,S_,...,NaN,3,184,185,5,flow=From-Botnet-V44-TCP-Attempt,150.35.87.168 -> 127.149.234.49,1.658903e+09,3,5
220699,220700,2022-07-27 06:24:41.499263,3.066254,udp,49.78.174.245,27883,<->,150.35.87.232,13360,CON,...,0.0,4,298,142,0,flow=Background-UDP-Established,49.78.174.245 -> 150.35.87.232,1.658903e+09,3,5
220700,220701,2022-07-27 06:24:41.651996,9.011096,tcp,150.35.87.168,3029,->,127.149.234.50,22,S_,...,NaN,2,184,185,5,flow=From-Botnet-V44-TCP-Attempt,150.35.87.168 -> 127.149.234.50,1.658903e+09,3,5
220701,220702,2022-07-27 06:24:41.720452,0.000226,udp,150.35.87.6,37917,<->,150.35.83.12,53,CON,...,0.0,2,249,78,1,flow=To-Background-UDP-CVUT-DNS-Server,150.35.87.6 -> 150.35.83.12,1.658903e+09,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278571,278572,2022-07-27 07:24:40.714187,1.676012,tcp,181.80.74.30,1962,->,150.35.90.55,3391,SRPA_SPA,...,0.0,193,145139,6153,0,flow=Background-TCP-Established,181.80.74.30 -> 150.35.90.55,1.658907e+09,3,5
278572,278573,2022-07-27 07:24:40.724799,1.643468,tcp,181.80.74.30,1964,->,150.35.90.52,3388,FSRPA_SRPA,...,0.0,223,176502,6842,0,flow=Background-TCP-Established,181.80.74.30 -> 150.35.90.52,1.658907e+09,3,5
278573,278574,2022-07-27 07:24:40.786840,1.038942,tcp,181.80.74.30,1995,->,150.35.89.91,3386,SRPA_SPA,...,0.0,204,162048,6479,0,flow=Background-TCP-Established,181.80.74.30 -> 150.35.89.91,1.658907e+09,3,5
278574,278575,2022-07-27 07:24:40.798059,0.000000,igmp,150.35.90.10,0,->,227.3.4.38,0,INT,...,NaN,0,60,60,0,flow=Background,150.35.90.10 -> 227.3.4.38,1.658907e+09,3,5


In [42]:
dfb = df.loc[df['Label'] == 5]
dfnm = df.loc[df['Label'] != 5]

In [43]:
# dfb.to_csv(f'data/{dt}_bot.csv')

In [44]:
# dfb[['StartTime','Conversation','Proto','LabelStr','Label','Window_lower','Window_upper_excl']].to_csv(f'data_/{dt}_bot.csv')
dfb[['StartTime','Conversation','Proto','LabelStr']].to_csv(f'data_/{dt}_bot.csv')

In [45]:
del df
del dfb
del dfnm